In [307]:
# your other imports here ...
import sys, os
import pandas as pd
import matplotlib.pyplot as plt

# TODO: replace with your path/to/ninarow
ninarowdir = os.path.dirname(os.getcwd())
modelfitdir = ninarowdir + "/model_fitting/"
# os.listdir(modelfitdir)

# sets the import path to the model-fitting directory
sys.path.insert(0, modelfitdir)
from parsers import *
from model_fit import *
from utils import *
import model_fit
from tqdm import tqdm

# WARNING: %load_ext autoreload and %autoreload 2 may interfere with 
# the Multi-threading processes!
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [308]:
from fourbynine import *

In [338]:
import xarray as xr

In [560]:
import xarray as xr

def get_mnk(): 
    return 4, 9, 4


def win_patterns(m=4, n=9, k=4, direction="-/\\|"):
     
    """
    Generate k-length patterns in a m x n grid in specified directions.

    Parameters:
    m (int): Number of rows in the grid. Default is 4.
    n (int): Number of columns in the grid. Default is 9.
    k (int): Length of the patterns to generate. Default is 4.
    direction (str): Directions to generate patterns. Can include any combination of 
                        '-', '|', '/', '\\'. Default is all directions "-/\\|".
                    For all cardinal directions, use "-|".

    Returns:
    xr.DataArray: A DataArray where each row represents a k-length pattern in the grid.
    the positions of the pattern and 0s elsewhere. The DataArray includes
    coordinates for direction and pattern keys.

    Notes:
    - The function generates horizontal, vertical, and diagonal patterns based on 
        the specified directions.
    - Horizontal patterns are generated if '-' is in direction.
    - Vertical patterns are generated if '|' is in direction.
    - Forward diagonal patterns (/) are generated if '/' is in direction.
    - Backward diagonal patterns (\\) are generated if '\\' is in direction.
    """
    
    directions = {
        "-": [np.arange(n * row + col, n * row + col + k) for row in range(m) for col in range(n - k + 1)],
        "|": [np.arange(n * row + col, n * (row + k - 1) + col + 1, n) for row in range(m - k + 1) for col in range(n)],
        "\\": [np.arange(n * row + col, n * row + col + (k - 1) * (n + 1) + 1, n + 1) for row in range(m - k + 1) for col in range(n - k + 1)],
        "/": [np.arange(n * row + col, n * row + col + (k - 1) * (n - 1) + 1, n - 1) for row in range(m - k + 1) for col in range(k - 1, n)]
    }

    patterns = [pattern for d in direction if d in directions for pattern in directions[d]]
    win_id = [f"{d}#{i}" for d in direction if d in directions for i in range(len(directions[d]))]
    directions = [d for d in direction if d in directions for i in range(len(directions[d]))]

    patterns = np.stack(patterns)
    win_array = np.zeros((len(patterns), m * n)).astype(int)
    win_array[np.arange(len(patterns)).reshape(len(patterns), -1), patterns] = 1

    # Create an xarray DataArray
    return pd.DataFrame({"direction": directions, "pattern": win_id, "position": [w for w in win_array]})

In [626]:
def make_feature_from_template(template, new_cols = {}, direction="-/\\|"):  
    wins = win_patterns(4, 9,  k=len(template), direction=direction)
    template = np.array(template)

    pieces = np.stack(wins["position"].values)
    pieces[pieces > 0] = np.tile(template, len(pieces, ))

    spaces = np.stack(wins["position"].values)
    spaces[spaces > 0] = np.tile(1 - template, len(spaces,))

    df = pd.DataFrame({"pieces": [p for p in pieces], "spaces": [s for s in spaces]})
    df["template"] = str(template)
    df = pd.concat([df, wins.drop("position", axis = 1)], axis=1)

    for col, val in new_cols.items():
        df[col] = val

    return df


def get_features(heuristic_groups): 
    features = []
    for group_name, templates in heuristic_groups.items():
        for template in templates:
            features.append(make_feature_from_template(template, new_cols = {"group": group_name}, direction="-/\\|"))
    
    features = pd.concat(features).reset_index(drop = True)
    features["n_pieces"] = features["pieces"].apply(lambda x: x.sum())
    features["n_spaces"] = features["spaces"].apply(lambda x: x.sum())
    return features

In [646]:
heuristic_groups = {
    "4iar":             [[1, 1, 1, 1]], 
    "3iarconn":         [[0, 1, 1, 1], [1, 1, 1, 0]], 
    "3iardisconn":      [[1, 0, 1, 1], [1, 1, 0, 1]], 
    "2iarconn":         [[1, 1, 0, 0], [0, 1, 1, 0], [0, 0, 1, 1]], 
    "2iardisconn":      [[1, 0, 0, 1], [1, 0, 1, 0], [0, 1, 0, 1]],
    "singleton":        [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
}

In [647]:
features = get_features(heuristic_groups)

In [817]:
bk = binary_array_to_int(np.array([0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
wh = binary_array_to_int(np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
n = 300000
board = np.stack([[bk, wh] * n

In [906]:
import time

bk = binary_array_to_int(np.array([0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
wh = binary_array_to_int(np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
n = 2000000
board = np.stack([[bk, wh]] * n)

start = time.time()
matched = features.drop(['pieces', 'spaces'], axis=1)

# Convert pieces and spaces to binary integers
pieces_int = np.array([int(''.join(map(str, p)), 2) for p in features.pieces.values])[:, np.newaxis]
spaces_int = np.array([int(''.join(map(str, s)), 2) for s in features.spaces.values])[:, np.newaxis]

# Convert player and opponent to binary integers
player_int = board[:, 0]
opponent_int = board[:, 1]

# Calculate filled positions using bitwise AND
player_filled = (pieces_int & player_int) == pieces_int
opponent_filled = (pieces_int  & opponent_int) == pieces_int 
spaces_filled = (spaces_int  & ~(player_int | opponent_int)) == spaces_int 

# # Calculate matches
# player_match = (player_filled & spaces_filled).T
# opponent_match = (opponent_filled & spaces_filled).T

end = time.time()
print(end - start)

56.271212100982666


In [909]:
import numpy as np
import pandas as pd
import time
from scipy.sparse import csc_matrix

bk = np.array([0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
wh = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
n = 82944
board = np.stack([[bk, wh]] * n)

# Convert pieces and spaces to binary integers
pieces_int = csc_matrix(np.stack(features.pieces.values).astype(int))
spaces_int = csc_matrix(np.stack(features.spaces.values).astype(int))

# Convert player and opponent to binary integers
player_int = csc_matrix(board[:, 0].astype(bool))
opponent_int = csc_matrix(board[:, 1].astype(bool))
spaces_int = csc_matrix((1 - (board[:, 0] + board[:, 1])).astype(bool))

start = time.time()

player_filled = (pieces_bool @ player_int.T) != features.n_pieces.values[:, np.newaxis]
opponent_filled = (pieces_bool @ opponent_int.T) != features.n_pieces.values[:, np.newaxis]
spaces_filled = (spaces_bool @ spaces_int.T) != features.n_spaces.values[:, np.newaxis]

player_match = player_filled & spaces_filled
opponent_match = opponent_filled & spaces_filled

end = time.time()
print(end - start)

1.2659730911254883


In [902]:
(pieces_bool @ player_int.T) 

<Compressed Sparse Row sparse matrix of dtype 'bool'
	with 20800000 stored elements and shape (675, 200000)>

In [908]:
64 * 36 * 36

82944

In [885]:
player_filled = (pieces_bool @ player_int.T) - csr_matrix(features.n_pieces.values[:, np.newaxis])

ValueError: inconsistent shapes

In [884]:
player_filled

False

In [600]:
def binary_array_to_int(binary_array):
    """
    Convert a 36-length binary numpy array into an integer by expressing it as a binary number.

    Parameters:
    binary_array (numpy.ndarray): A numpy array of length 36 containing binary values (0s and 1s).

    Returns:
    int: The integer representation of the binary array.
    """
    if len(binary_array) != 36:
        raise ValueError("The input array must have a length of 36.")
    
    binary_str = ''.join(binary_array.astype(int).astype(str))
    return int(binary_str, 2)

def int_to_binary_array(integer, length=36):
    """
    Convert an integer into a binary numpy array of specified length.

    Parameters:
    integer (int or str): The integer to convert. If a string is provided, it will be cast to an integer.
    length (int): The length of the resulting binary array. Default is 36.

    Returns:
    numpy.ndarray: A numpy array of the specified length containing binary values (0s and 1s).
    """
    if isinstance(integer, str):
        integer = int(integer)
    binary_str = bin(integer)[2:].zfill(length)
    return np.array(list(binary_str), dtype=int)

def binary_string_to_binary_array(binary_string, length=36):
    """
    Convert a binary string into a binary numpy array of specified length.

    Parameters:
    binary_string (str): The binary string to convert.
    length (int): The length of the resulting binary array. Default is 36.

    Returns:
    numpy.ndarray: A numpy array of the specified length containing binary values (0s and 1s).
    """
    return np.array(list(binary_string.zfill(length)), dtype=int)

def plot_board(board): 
    black = board.get_pieces(0).to_string()
    white = board.get_pieces(1).to_string()

    black = binary_string_to_binary_array(black, length=36).reshape(4, 9)
    white = binary_string_to_binary_array(white, length=36).reshape(4, 9)

    plt.imshow(white - black, cmap='gray', interpolation='none')

def plot_feature(feature):
    """
    Plot a feature using imshow, setting feature.pieces to 1, feature.spaces to -1, and all else to np.nan.

    Parameters:
    feature (fourbynine.fourbynine_heuristic_feature): The feature to plot.
    """
    pieces = binary_string_to_binary_array(feature.pieces.to_string(), length=36).reshape(4, 9)
    spaces = binary_string_to_binary_array(feature.spaces.to_string(), length=36).reshape(4, 9)
    
    plot_array = np.full(pieces.shape, np.nan)
    plot_array[pieces == 1] = 1
    plot_array[spaces == 1] = -1
    
    plt.imshow(plot_array, cmap='coolwarm', interpolation='none')
    plt.show()

def make_feature_from_binary_array(pieces, spaces, min_empty):
    return fourbynine_heuristic_feature(fourbynine_pattern(binary_array_to_int(pieces)), fourbynine_pattern(binary_array_to_int(spaces)), int(min_empty))

# Example usage:
binary_array = np.array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])
integer_value = binary_array_to_int(binary_array)
print(integer_value)

1074791426


In [459]:
heuristic_groups = {
    "4iar":             [[1, 1, 1, 1]], 
    "3iarconn":         [[0, 1, 1, 1], [1, 1, 1, 0]], 
    "3iardisconn":      [[1, 0, 1, 1], [1, 1, 0, 1]], 
}

groups = build_all_feature_groups_concat(heuristic_groups)


In [460]:
groups[1]

<xarray.DataArray (group: 2, type: 2, win_config: 45, position: 36)> Size: 52kB
array([[[[0., 1., 1., ..., 0., 0., 0.],
         [0., 0., 1., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 1., 0., 0.],
         [0., 0., 0., ..., 0., 1., 0.],
         [0., 0., 0., ..., 0., 0., 1.]],

        [[1., 0., 0., ..., 0., 0., 0.],
         [0., 1., 0., ..., 0., 0., 0.],
         [0., 0., 1., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]],


       [[[1., 1., 1., ..., 0., 0., 0.],
         [0., 1., 1., ..., 0., 0., 0.],
         [0., 0., 1., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 1., 0., 0.],
         [0., 0., 0., ..., 0., 1., 0.],
         [0., 0., 0., ..., 0., 0., 1.]]]])
Coordinates:
  * type        (type) <U7 56B 'pieces' 'empties'
    direction   (win_config) <U1 180B '-' '-' '-' '-' '-' ... '|' '|' '|' '|'
  * win_config  (win_config) <U4 720B '-#0' '-#1' '-#2' ... '|#6' '|#7' '|#8'
  * position    (position) int64 288B 0 1 2 3 4 5 6 7 ... 29 30 31 32 33 34 35
  * group       (group) <U8 64B '3iarconn' '3iarconn'

In [311]:
features, feature_names = generate_all_feature_configurations([[0, 1, 1, 1], [1, 0, 1, 1], [1, 1, 0, 1], [1, 1, 1, 0]], "3-in-a-row")

# for feature, feature_name in zip(features, feature_names):
#     print(feature_name)
#     plot_feature(feature)

In [ ]:
board = fourbynine_board(fourbynine_pattern(bk), fourbynine_pattern(wh))

In [336]:
for group in heuristic_groups: 
    features, feature_names = generate_all_feature_configurations(heuristic_groups[group], group)
    for feature, feature_name in zip(features, feature_names):
        if feature.contained_in(board, board.active_player()):
            print(f"Board contains feature: {feature_name}")

Board contains feature: 2iarconn.[1 1 0 0].494.d-.n2
Board contains feature: 2iarconn.[0 1 1 0].494.d-.n1
Board contains feature: 2iarconn.[0 0 1 1].494.d-.n0


In [ ]:
for group in heuristic_groups: 
    for feature, feature_name in zip(features, feature_names):
        print(feature_name)
        plot_feature(feature)

In [314]:
evaluator = fourbynine_feature_evaluator()
evaluated = evaluator.query_spaces(board)
np.array(evaluated)

array(<Swig Object of type 'std::vector< std::size_t,std::allocator< std::size_t > > *' at 0x122bbf5a0>,
      dtype=object)